<a href="https://colab.research.google.com/github/Varchala/Secure_Private_AI/blob/main/Split_Learning_varchala_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft==0.2.9 --quiet

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import syft as sy
import torch
from torchvision import datasets, transforms  # it may raise errors, and you need restart the runtime
from torch import nn, optim
import syft as sy
import numpy as np

In [3]:
import torch
import syft as sy

# allow pysyft to work its magic on torch tensors
hook = sy.TorchHook(torch)

# create a virtual worker. in an actual setting this would be on a different machine
client = sy.VirtualWorker( hook, id='client' )

# define a tensor and send it to the client
x = torch.tensor([1,2,3,4,5])
# this leaves us with a pointer to the tensor
x_pointer = x.send( client )

# check out some meta data
print( x_pointer )
print( client._objects )

# we can use this pointers like normal tensors
result = x_pointer + x_pointer
print( result )

# if we want the result we can call get() to send the tensor back to us
result_local = result.get()
# once we call get() it removes the tensor from the other side and our pointer
# becomes invalid
print( result_local )
print( client._objects )
# print( result )

(Wrapper)>[PointerTensor | me:77340015578 -> client:18337107945]
{18337107945: tensor([1, 2, 3, 4, 5])}
(Wrapper)>[PointerTensor | me:50033037401 -> client:28915619493]
tensor([ 2,  4,  6,  8, 10])
{18337107945: tensor([1, 2, 3, 4, 5])}


In [4]:
# Data preprocessing
hook = sy.TorchHook(torch)
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.CIFAR100('cifar100', download=True, train=True, transform=transform)

Files already downloaded and verified


In [5]:
trainset.data.shape,len(trainset.targets)

((50000, 32, 32, 3), 50000)

In [6]:
def extract_n_classes( data, labels,n):
    data_cl=[]
    y=[]
    for i in range(n):
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))


    x = np.vstack( (data_cl) )
    y = np.array(y)
    return x, y

In [7]:
x_train,y_train = extract_n_classes(trainset.data,np.array(trainset.targets),n=100)

In [8]:
x_train.shape,y_train.shape

((50000, 32, 32, 3), (50000,))

In [9]:
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 5

In [10]:
print(y_train)

[ 0  0  0 ... 99 99 99]


In [11]:
trainset.data = x_train
trainset.targets = y_train

In [12]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False)

torch.manual_seed(0)

In [13]:
trainloader.dataset.targets

array([ 0,  0,  0, ..., 99, 99, 99])

In [14]:
class SplitNN:
    def __init__(self, models, optimizers):
        self.models = models
        self.optimizers = optimizers
        
    def forward(self, x):
        a = []
        remote_a = []
        
        a.append(models[0](x))
        if a[-1].location == models[1].location:
            remote_a.append(a[-1].detach().requires_grad_())
        else:
            remote_a.append(a[-1].detach().move(models[1].location).requires_grad_())

        i=1    
        while i < (len(models)-1):
            
            a.append(models[i](remote_a[-1]))
            if a[-1].location == models[i+1].location:
                remote_a.append(a[-1].detach().requires_grad_())
            else:
                remote_a.append(a[-1].detach().move(models[i+1].location).requires_grad_())
            
            i+=1
        
        a.append(models[i](remote_a[-1]))
        self.a = a
        self.remote_a = remote_a
        
        return a[-1]
    
    def backward(self):
        a=self.a
        remote_a=self.remote_a
        optimizers = self.optimizers
        
        i= len(models)-2   
        while i > -1:
            if remote_a[i].location == a[i].location:
                grad_a = remote_a[i].grad.copy()
            else:
                grad_a = remote_a[i].grad.copy().move(a[i].location)
            a[i].backward(grad_a)
            i-=1

    
    def zero_grads(self):
        for opt in optimizers:
            opt.zero_grad()
        
    def step(self):
        for opt in optimizers:
            opt.step()
            

# Define our model segments

In [15]:

total = 100

input_size = 3072
hidden_sizes = [128, 640]
output_size = 100

layers = [nn.Linear(input_size, hidden_sizes[0]), nn.ReLU()]

for i in range(1, int(total/2)-1):
  if (i%2 != 0):
    layers.extend([nn.Linear(hidden_sizes[0], hidden_sizes[1]), nn.ReLU()])
  else:
    layers.extend([nn.Linear(hidden_sizes[1], hidden_sizes[0]), nn.ReLU()])

# the final layer
layers.extend([nn.Linear(hidden_sizes[0], output_size), nn.LogSoftmax(dim=1)])

print(len(layers))
layers

100


[Linear(in_features=3072, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=640, bias=True),
 ReLU(),
 Linear(in_features=640, out_features=128, bias=True),
 ReLU(),
 Linear(in_features=128, out_features=6

# Each worker has only one class

# 10 Clients

In [16]:
epochs = 2
models = []
i=0
while(i<100):
    models.append(nn.Sequential(*layers[i:i+10]))
    i += 10

In [17]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_features=128, bias=True)
   (5): ReLU()
   (6): Linear(in_features=128, out_features=640, bias=True)
   (7): ReLU()
   (8): Linear(in_features=640, out_features=128, bias=True)
   (9): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
   (2): Linear(in_features=640, out_features=128, bias=True)
   (3): ReLU()
   (4): Linear(in_features=128, out_features=640, bias=True)
   (5): ReLU()
   (6): Linear(in_features=640, out_features=128, bias=True)
   (7): ReLU()
   (8): Linear(in_features=128, out_features=640, bias=True)
   (9): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_featur

In [18]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [19]:

# create some workers
alices = []
for i in range(10):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [20]:
workers

(<VirtualWorker id:alice1 #objects:0>,
 <VirtualWorker id:alice2 #objects:0>,
 <VirtualWorker id:alice3 #objects:0>,
 <VirtualWorker id:alice4 #objects:0>,
 <VirtualWorker id:alice5 #objects:0>,
 <VirtualWorker id:alice6 #objects:0>,
 <VirtualWorker id:alice7 #objects:0>,
 <VirtualWorker id:alice8 #objects:0>,
 <VirtualWorker id:alice9 #objects:0>,
 <VirtualWorker id:alice10 #objects:0>)

In [21]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [22]:
splitNN =  SplitNN(models, optimizers)

In [23]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [24]:
def test(x, target, splitNN):
    
    #1) Zero our grads
    # splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    
    #3) Figure out how much we missed by
    # criterion = nn.NLLLoss()
    # loss = criterion(pred, target)
    
    
    return pred

In [25]:
models[-1].location.id

'alice10'

In [26]:
%%time
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.026189804077148 Training Accuracy: 0.07217071611253197
Epoch 1 - Training loss: 3.626110553741455 Training Accuracy: 0.1257992327365729


In [27]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [28]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [29]:
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01


# 20 Clients

In [15]:
100/20

5.0

In [16]:
epochs = 3
models = []
i=0
while(i<100):
    models.append(nn.Sequential(*layers[i:i+5]))
    i += 5

In [17]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_features=128, bias=True)
 ), Sequential(
   (0): ReLU()
   (1): Linear(in_features=128, out_features=640, bias=True)
   (2): ReLU()
   (3): Linear(in_features=640, out_features=128, bias=True)
   (4): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
   (2): Linear(in_features=640, out_features=128, bias=True)
   (3): ReLU()
   (4): Linear(in_features=128, out_features=640, bias=True)
 ), Sequential(
   (0): ReLU()
   (1): Linear(in_features=640, out_features=128, bias=True)
   (2): ReLU()
   (3): Linear(in_features=128, out_features=640, bias=True)
   (4): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Li

In [18]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [19]:

# create some workers
alices = []
for i in range(20):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [20]:
workers

(<VirtualWorker id:alice1 #objects:0>,
 <VirtualWorker id:alice2 #objects:0>,
 <VirtualWorker id:alice3 #objects:0>,
 <VirtualWorker id:alice4 #objects:0>,
 <VirtualWorker id:alice5 #objects:0>,
 <VirtualWorker id:alice6 #objects:0>,
 <VirtualWorker id:alice7 #objects:0>,
 <VirtualWorker id:alice8 #objects:0>,
 <VirtualWorker id:alice9 #objects:0>,
 <VirtualWorker id:alice10 #objects:0>,
 <VirtualWorker id:alice11 #objects:0>,
 <VirtualWorker id:alice12 #objects:0>,
 <VirtualWorker id:alice13 #objects:0>,
 <VirtualWorker id:alice14 #objects:0>,
 <VirtualWorker id:alice15 #objects:0>,
 <VirtualWorker id:alice16 #objects:0>,
 <VirtualWorker id:alice17 #objects:0>,
 <VirtualWorker id:alice18 #objects:0>,
 <VirtualWorker id:alice19 #objects:0>,
 <VirtualWorker id:alice20 #objects:0>)

In [21]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [22]:
splitNN =  SplitNN(models, optimizers)

In [23]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [24]:
def test(x, target, splitNN):
    
    pred = splitNN.forward(x)
    
    return pred

In [25]:
models[-1].location.id

'alice20'

In [26]:
%%time
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.026189804077148 Training Accuracy: 0.07217071611253197
Epoch 1 - Training loss: 3.626110553741455 Training Accuracy: 0.1257992327365729
Epoch 2 - Training loss: 3.262223482131958 Training Accuracy: 0.20076726342711
CPU times: user 13min 51s, sys: 3min 51s, total: 17min 43s
Wall time: 17min 45s


In [27]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [28]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [29]:
# loss=0
%%time
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
CPU times: user 14min 18s, sys: 43.9 s, total: 15min 1s
Wall time: 15min


# 50 Clients

In [15]:
epochs = 3
models = []
i=0
while(i<100):
    models.append(nn.Sequential(*layers[i:i+2]))
    i += 2

In [16]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [17]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [18]:
workers = []
# create some workers
alices = []
for i in range(50):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [19]:
workers

(<VirtualWorker id:alice1 #objects:0>,
 <VirtualWorker id:alice2 #objects:0>,
 <VirtualWorker id:alice3 #objects:0>,
 <VirtualWorker id:alice4 #objects:0>,
 <VirtualWorker id:alice5 #objects:0>,
 <VirtualWorker id:alice6 #objects:0>,
 <VirtualWorker id:alice7 #objects:0>,
 <VirtualWorker id:alice8 #objects:0>,
 <VirtualWorker id:alice9 #objects:0>,
 <VirtualWorker id:alice10 #objects:0>,
 <VirtualWorker id:alice11 #objects:0>,
 <VirtualWorker id:alice12 #objects:0>,
 <VirtualWorker id:alice13 #objects:0>,
 <VirtualWorker id:alice14 #objects:0>,
 <VirtualWorker id:alice15 #objects:0>,
 <VirtualWorker id:alice16 #objects:0>,
 <VirtualWorker id:alice17 #objects:0>,
 <VirtualWorker id:alice18 #objects:0>,
 <VirtualWorker id:alice19 #objects:0>,
 <VirtualWorker id:alice20 #objects:0>,
 <VirtualWorker id:alice21 #objects:0>,
 <VirtualWorker id:alice22 #objects:0>,
 <VirtualWorker id:alice23 #objects:0>,
 <VirtualWorker id:alice24 #objects:0>,
 <VirtualWorker id:alice25 #objects:0>,
 <Virtual

In [20]:
# Send Model Segments to starting locations
# model_locations = []
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [21]:
splitNN =  SplitNN(models, optimizers)

In [22]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [23]:
def test(x, target, splitNN):
    
    pred = splitNN.forward(x)
    
    return pred

In [24]:
models[-1].location.id

'alice50'

In [25]:
%%time
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.026189804077148 Training Accuracy: 0.07217071611253197
Epoch 1 - Training loss: 3.626110553741455 Training Accuracy: 0.1257992327365729
Epoch 2 - Training loss: 3.262223482131958 Training Accuracy: 0.20076726342711
CPU times: user 21min 31s, sys: 6min 31s, total: 28min 2s
Wall time: 28min 4s


In [26]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [27]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [28]:
%%time
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
CPU times: user 29min 37s, sys: 17min 40s, total: 47min 17s
Wall time: 47min 14s


# 100 Clients

In [ ]:

epochs = 2

# Define our model segments
models = []
input_size = 3072
hidden_sizes = [128, 640]
output_size = 100
inp = input_size
out = hidden_sizes[0]
k=0
for i in range(100):

                models.append(nn.Sequential(
                            nn.Linear(inp, out),
                            nn.ReLU(),
                ))
                inp = hidden_sizes[k]
                if k==0:
                    k=1
                else:
                    k=0
                out = hidden_sizes[k]
models.append(nn.Sequential(
                nn.Linear(inp, output_size),
                nn.LogSoftmax(dim=1)
    ))



In [ ]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [ ]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [ ]:

# create some workers
alices = []
for i in range(101):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [ ]:
workers

(<VirtualWorker id:alice1 #objects:6>,
 <VirtualWorker id:alice2 #objects:6>,
 <VirtualWorker id:alice3 #objects:6>,
 <VirtualWorker id:alice4 #objects:6>,
 <VirtualWorker id:alice5 #objects:6>,
 <VirtualWorker id:alice6 #objects:6>,
 <VirtualWorker id:alice7 #objects:6>,
 <VirtualWorker id:alice8 #objects:6>,
 <VirtualWorker id:alice9 #objects:6>,
 <VirtualWorker id:alice10 #objects:6>,
 <VirtualWorker id:alice11 #objects:6>,
 <VirtualWorker id:alice12 #objects:6>,
 <VirtualWorker id:alice13 #objects:6>,
 <VirtualWorker id:alice14 #objects:6>,
 <VirtualWorker id:alice15 #objects:6>,
 <VirtualWorker id:alice16 #objects:6>,
 <VirtualWorker id:alice17 #objects:6>,
 <VirtualWorker id:alice18 #objects:6>,
 <VirtualWorker id:alice19 #objects:6>,
 <VirtualWorker id:alice20 #objects:6>,
 <VirtualWorker id:alice21 #objects:6>,
 <VirtualWorker id:alice22 #objects:6>,
 <VirtualWorker id:alice23 #objects:6>,
 <VirtualWorker id:alice24 #objects:6>,
 <VirtualWorker id:alice25 #objects:6>,
 <Virtual

In [ ]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [ ]:
def test(x, target, splitNN):
    
    pred = splitNN.forward(x)
    
    return pred

In [ ]:
models[-1].location.id

'alice101'

In [ ]:
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 3.7677416801452637 Training Accuracy: 0.12228260869565218
Epoch 1 - Training loss: 3.2349305152893066 Training Accuracy: 0.2108375959079284


In [ ]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [ ]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [ ]:
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01


# Every worker has all the classes

In [16]:
x_new = []
y_new = []
for i in range(500):
    for j in range(x_train.shape[0]//500):
        x_new.append(x_train[j*500+i])
        y_new.append(y_train[j*500+i])

In [17]:
trainset.data = np.array(x_new)
trainset.targets = np.array(y_new)

In [18]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False)

torch.manual_seed(0)

In [19]:
trainloader.dataset.targets

array([ 0,  1,  2, ..., 97, 98, 99])

# 10 Clients

In [20]:
epochs = 2
models = []
i=0
while(i<100):
    models.append(nn.Sequential(*layers[i:i+10]))
    i += 10

In [21]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_features=128, bias=True)
   (5): ReLU()
   (6): Linear(in_features=128, out_features=640, bias=True)
   (7): ReLU()
   (8): Linear(in_features=640, out_features=128, bias=True)
   (9): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
   (2): Linear(in_features=640, out_features=128, bias=True)
   (3): ReLU()
   (4): Linear(in_features=128, out_features=640, bias=True)
   (5): ReLU()
   (6): Linear(in_features=640, out_features=128, bias=True)
   (7): ReLU()
   (8): Linear(in_features=128, out_features=640, bias=True)
   (9): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
   (2): Linear(in_features=128, out_features=640, bias=True)
   (3): ReLU()
   (4): Linear(in_features=640, out_featur

In [22]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [23]:

# create some workers
alices = []
for i in range(11):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [24]:
workers

(<VirtualWorker id:alice1 #objects:0>,
 <VirtualWorker id:alice2 #objects:0>,
 <VirtualWorker id:alice3 #objects:0>,
 <VirtualWorker id:alice4 #objects:0>,
 <VirtualWorker id:alice5 #objects:0>,
 <VirtualWorker id:alice6 #objects:0>,
 <VirtualWorker id:alice7 #objects:0>,
 <VirtualWorker id:alice8 #objects:0>,
 <VirtualWorker id:alice9 #objects:0>,
 <VirtualWorker id:alice10 #objects:0>,
 <VirtualWorker id:alice11 #objects:0>)

In [25]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [26]:
splitNN =  SplitNN(models, optimizers)

In [27]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [28]:
def test(x, target, splitNN):
    
    #1) Zero our grads
    # splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    
    #3) Figure out how much we missed by
    # criterion = nn.NLLLoss()
    # loss = criterion(pred, target)
    
    
    return pred

In [29]:
models[-1].location.id

'alice10'

In [31]:
%%time
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.6063055992126465 Training Accuracy: 0.009990409207161126
Epoch 1 - Training loss: 4.605844497680664 Training Accuracy: 0.009990409207161126
CPU times: user 8min 9s, sys: 2min 15s, total: 10min 25s
Wall time: 10min 25s


In [ ]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [ ]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [ ]:
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.0107


# 20 Clients

In [ ]:

epochs = 10



# Define our model segments
models = []
input_size = 3072
hidden_sizes = [128, 640]
output_size = 100
inp = input_size
out = hidden_sizes[0]
k=0
for i in range(20):

                models.append(nn.Sequential(
                            nn.Linear(inp, out),
                            nn.ReLU(),
                ))
                inp = hidden_sizes[k]
                if k==0:
                    k=1
                else:
                    k=0
                out = hidden_sizes[k]
models.append(nn.Sequential(
                nn.Linear(inp, output_size),
                nn.LogSoftmax(dim=1)
    ))



In [ ]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [ ]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [ ]:

# create some workers
alices = []
for i in range(21):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [ ]:
workers

(<VirtualWorker id:alice1 #objects:24>,
 <VirtualWorker id:alice2 #objects:24>,
 <VirtualWorker id:alice3 #objects:24>,
 <VirtualWorker id:alice4 #objects:24>,
 <VirtualWorker id:alice5 #objects:24>,
 <VirtualWorker id:alice6 #objects:24>,
 <VirtualWorker id:alice7 #objects:24>,
 <VirtualWorker id:alice8 #objects:24>,
 <VirtualWorker id:alice9 #objects:24>,
 <VirtualWorker id:alice10 #objects:24>,
 <VirtualWorker id:alice11 #objects:26>,
 <VirtualWorker id:alice12 #objects:14>,
 <VirtualWorker id:alice13 #objects:14>,
 <VirtualWorker id:alice14 #objects:14>,
 <VirtualWorker id:alice15 #objects:14>,
 <VirtualWorker id:alice16 #objects:14>,
 <VirtualWorker id:alice17 #objects:14>,
 <VirtualWorker id:alice18 #objects:14>,
 <VirtualWorker id:alice19 #objects:14>,
 <VirtualWorker id:alice20 #objects:14>,
 <VirtualWorker id:alice21 #objects:14>)

In [ ]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [ ]:
def test(x, target, splitNN):
    
    pred = splitNN.forward(x)
    
    return pred

In [ ]:
models[-1].location.id

'alice21'

In [ ]:
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

Epoch 0 - Training loss: 4.605624675750732 Training Accuracy: 0.009990409207161126
Epoch 1 - Training loss: 4.6054511070251465 Training Accuracy: 0.009990409207161126
Epoch 2 - Training loss: 4.605381488800049 Training Accuracy: 0.009990409207161126
Epoch 3 - Training loss: 4.605358600616455 Training Accuracy: 0.009910485933503837
Epoch 4 - Training loss: 4.6053466796875 Training Accuracy: 0.010050351662404092
Epoch 5 - Training loss: 4.605339050292969 Training Accuracy: 0.010050351662404092
Epoch 6 - Training loss: 4.605331897735596 Training Accuracy: 0.009810581841432225
Epoch 7 - Training loss: 4.605337619781494 Training Accuracy: 0.00941096547314578
Epoch 8 - Training loss: 4.605342864990234 Training Accuracy: 0.00941096547314578
Epoch 9 - Training loss: 4.605345249176025 Training Accuracy: 0.008951406649616368


In [ ]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [ ]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [ ]:
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01


# 50 Clients

In [ ]:

epochs = 10



# Define our model segments
models = []
input_size = 3072
hidden_sizes = [128, 640]
output_size = 100
inp = input_size
out = hidden_sizes[0]
k=0
for i in range(50):

                models.append(nn.Sequential(
                            nn.Linear(inp, out),
                            nn.ReLU(),
                ))
                inp = hidden_sizes[k]
                if k==0:
                    k=1
                else:
                    k=0
                out = hidden_sizes[k]
models.append(nn.Sequential(
                nn.Linear(inp, output_size),
                nn.LogSoftmax(dim=1)
    ))



In [ ]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [ ]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [ ]:

# create some workers
alices = []
for i in range(51):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [ ]:
workers

(<VirtualWorker id:alice1 #objects:32>,
 <VirtualWorker id:alice2 #objects:32>,
 <VirtualWorker id:alice3 #objects:32>,
 <VirtualWorker id:alice4 #objects:32>,
 <VirtualWorker id:alice5 #objects:32>,
 <VirtualWorker id:alice6 #objects:32>,
 <VirtualWorker id:alice7 #objects:32>,
 <VirtualWorker id:alice8 #objects:32>,
 <VirtualWorker id:alice9 #objects:32>,
 <VirtualWorker id:alice10 #objects:32>,
 <VirtualWorker id:alice11 #objects:32>,
 <VirtualWorker id:alice12 #objects:24>,
 <VirtualWorker id:alice13 #objects:24>,
 <VirtualWorker id:alice14 #objects:24>,
 <VirtualWorker id:alice15 #objects:24>,
 <VirtualWorker id:alice16 #objects:24>,
 <VirtualWorker id:alice17 #objects:24>,
 <VirtualWorker id:alice18 #objects:24>,
 <VirtualWorker id:alice19 #objects:24>,
 <VirtualWorker id:alice20 #objects:24>,
 <VirtualWorker id:alice21 #objects:26>,
 <VirtualWorker id:alice22 #objects:14>,
 <VirtualWorker id:alice23 #objects:14>,
 <VirtualWorker id:alice24 #objects:14>,
 <VirtualWorker id:alice2

In [ ]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [ ]:
def test(x, target, splitNN):
    
    pred = splitNN.forward(x)
    
    return pred

In [ ]:
models[-1].location.id

'alice51'

In [ ]:
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

In [ ]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

In [ ]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [ ]:
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

# 100 Clients

In [ ]:

epochs = 2

# Define our model segments
models = []
input_size = 3072
hidden_sizes = [128, 640]
output_size = 100
inp = input_size
out = hidden_sizes[0]
k=0
for i in range(100):

                models.append(nn.Sequential(
                            nn.Linear(inp, out),
                            nn.ReLU(),
                ))
                inp = hidden_sizes[k]
                if k==0:
                    k=1
                else:
                    k=0
                out = hidden_sizes[k]
models.append(nn.Sequential(
                nn.Linear(inp, output_size),
                nn.LogSoftmax(dim=1)
    ))



In [ ]:
models

[Sequential(
   (0): Linear(in_features=3072, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=128, out_features=640, bias=True)
   (1): ReLU()
 ), Sequential(
   (0): Linear(in_features=640, out_features=128, bias=True)
   (1

In [ ]:
# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]


In [ ]:

# create some workers
alices = []
for i in range(101):
    alices.append(sy.VirtualWorker(hook, id="alice"+str(i+1)))

workers = tuple(alices)

In [ ]:
workers

(<VirtualWorker id:alice1 #objects:6>,
 <VirtualWorker id:alice2 #objects:6>,
 <VirtualWorker id:alice3 #objects:6>,
 <VirtualWorker id:alice4 #objects:6>,
 <VirtualWorker id:alice5 #objects:6>,
 <VirtualWorker id:alice6 #objects:6>,
 <VirtualWorker id:alice7 #objects:6>,
 <VirtualWorker id:alice8 #objects:6>,
 <VirtualWorker id:alice9 #objects:6>,
 <VirtualWorker id:alice10 #objects:6>,
 <VirtualWorker id:alice11 #objects:6>,
 <VirtualWorker id:alice12 #objects:6>,
 <VirtualWorker id:alice13 #objects:6>,
 <VirtualWorker id:alice14 #objects:6>,
 <VirtualWorker id:alice15 #objects:6>,
 <VirtualWorker id:alice16 #objects:6>,
 <VirtualWorker id:alice17 #objects:6>,
 <VirtualWorker id:alice18 #objects:6>,
 <VirtualWorker id:alice19 #objects:6>,
 <VirtualWorker id:alice20 #objects:6>,
 <VirtualWorker id:alice21 #objects:6>,
 <VirtualWorker id:alice22 #objects:6>,
 <VirtualWorker id:alice23 #objects:6>,
 <VirtualWorker id:alice24 #objects:6>,
 <VirtualWorker id:alice25 #objects:6>,
 <Virtual

In [ ]:
# Send Model Segments to starting locations
model_locations = alices

for model, location in zip(models, model_locations):
    model.send(location)

In [ ]:
splitNN =  SplitNN(models, optimizers)

In [ ]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
    # print(pred)
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)
  
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss,pred

In [ ]:
def test(x, target, splitNN):
    
    pred = splitNN.forward(x)
    
    return pred

In [ ]:
models[-1].location.id

'alice101'

In [ ]:
for i in range(epochs):
    running_loss = 0
    acc = 0
    for images, labels in trainloader:
        loss = 0
        acc_c = 0
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        loss,pred = train(images, labels, splitNN)
        # print(pred.shape)
        # print(labels.shape)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        acc_c = train_acc/len(labels)
        # print(train_acc.get()) 
        running_loss += loss.get()
        acc +=acc_c

    else:
        print("Epoch {} - Training loss: {} Training Accuracy: {}".format(i, running_loss/len(trainloader),acc/len(trainloader)))

In [ ]:
testset = datasets.CIFAR100('cifar100', download=True, train=False, transform=transform)

Files already downloaded and verified


In [ ]:
testloader = torch.utils.data.DataLoader(testset,shuffle=False)

torch.manual_seed(0)

### Testing the models

In [ ]:
# loss=0
acc_c = 0
acc = 0
with torch.no_grad():
 for images, labels in testloader:
        images = images.send(models[0].location)
        images = images.view(images.shape[0], -1)
        labels = labels.send(models[-1].location)
        pred = test(images, labels, splitNN)
        pred = torch.argmax(pred, dim=1)
        train_acc = torch.sum(pred == labels).get().item()
        # acc_c = train_acc/len(labels)
        # print(images.shape,labels.shape)
        acc +=train_acc
        # running_loss += loss.get()
print("Testing accuracy: {}".format(acc/len(testloader)))

Testing accuracy: 0.01
